In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
# plt.rcParams.update({'font.size': 18})
import seaborn as sns
import regex as re
import os
import numpy as np

from ast import literal_eval

In [ ]:
plt.rcParams.update({'font.size': 24})

In [ ]:
result_paths = [
    "/mnt/extra/continuous-training/results_fixed/1717700456995308991/",
    "/mnt/extra/continuous-training/results_fixed/1717444793172840349/",
    "/mnt/extra/continuous-training/results_fixed/1717705984940495539/",
    "/mnt/extra/continuous-training/results_fixed/1717575413839183617/",
    # "/mnt/extra/continuous-training/results_fixed/1717467553842856231/",
    "/mnt/extra/continuous-training/results/1718057739541878219",
    "/mnt/extra/continuous-training/results_fixed/1717575369578062901/",
    "/mnt/extra/continuous-training/results_fixed/1717579388065196179/",
    "/mnt/extra/continuous-training/results_fixed/1717701022363026467/",
    "/mnt/extra/continuous-training/results_fixed/1717608769994081703/",
    "/mnt/extra/continuous-training/results_fixed/1717579439091810476/",
    "/mnt/extra/continuous-training/results_fixed/1717587649593441377/",
    "/mnt/extra/continuous-training/results_fixed/1717700529336249757/",
    "/mnt/extra/continuous-training/results_fixed/1717635454957034095/",
    # "/mnt/extra/continuous-training/results_fixed/1717446231823791885/",
    "/mnt/extra/continuous-training/results_fixed/1717448773806529718/",
    "/mnt/extra/continuous-training/results_fixed/1717612290915437273/",
    "/mnt/extra/continuous-training/results_fixed/1717594617566228467/",
    "/mnt/extra/continuous-training/results_fixed/1717478686265173993/",
    "/mnt/extra/continuous-training/results_fixed/1717619549666300929/",
    "/mnt/extra/continuous-training/results_fixed/1717704913915861438/",
    # "/mnt/extra/continuous-training/results_fixed/1717456543236185969/",
    "/mnt/extra/continuous-training/results/1718057735112524876",
    "/mnt/extra/continuous-training/results_fixed/1717715829899615491/",
]

In [ ]:
stats_df = pd.DataFrame(columns=['folder_path', 'algo', 'param_path', 'csv_path', 'latencies_path', 'num_retrain', 'training_time_total', 'training_time_single', 'training_cpu_time', 'training_counter', 'inference_time_total', 'inference_time_single', 'inference_cpu_time', 'inference_counter'])
for stat in result_paths:
    contents = []
    with open(os.path.join(stat, 'parameters.txt'), "r") as f:
        contents = f.readlines()
    # Iterate to get variables
    folder_path = stat
    algo_name = ''
    csv_path = ''
    latencies_path = os.path.join(folder_path, 'latencies.csv')
    param_path = stat
    valid_entry = False
    retrained = -1
    training_time_total = -1
    training_time_single = -1
    training_cpu_time = -1
    training_counter = -1
    inference_time_total = -1
    inference_time_single = -1
    inference_cpu_time = -1
    inference_counter = -1
    for content in contents:
        if not valid_entry and 'Inference' in content:
            valid_entry = True
        if '-output' in content:
            try:
                algo_name = content.split(' = ')[1].split('\n')[0]
            except:
                algo_name = content.split('=')[1].split('\n')[0]
            csv_path = os.path.join(algo_name+'.csv')
        if 'Retrained' in content:
            retrained = float(content.split(' = ')[1].split('\n')[0])
        if 'Training time total' in content:
            training_time_total = float(content.split(' = ')[1].split(' s')[0])
        if 'Training time single' in content:
            training_time_single = float(content.split(' = ')[1].split(' s')[0])
        if 'Training CPU times usage' in content:
            training_cpu_time = float(content.split(' = ')[1].split(' sCPU')[0])
        if 'Training counter' in content:
            training_counter = float(content.split(' = ')[1].split('\n')[0])
        if 'Inference time total' in content:
            inference_time_total = float(content.split(' = ')[1].split(' s')[0])
        if 'Inference time single' in content:
            inference_time_single = float(content.split(' = ')[1].split(' s')[0])
        if 'Inference CPU times usage' in content:
            inference_cpu_time = float(content.split(' = ')[1].split(' sCPU')[0])
        if 'Inference counter' in content:
            inference_counter = float(content.split(' = ')[1].split('\n')[0])
    if valid_entry:
        stats_df.loc[len(stats_df)] = [folder_path, algo_name, param_path, csv_path, latencies_path, retrained, training_time_total, training_time_single, training_cpu_time, training_counter, inference_time_total, inference_time_single, inference_cpu_time, inference_counter]
stats_df

In [ ]:
latencies_acc_list = {}
latencies_rej_list = {}

values_df_1 = pd.DataFrame({
    'path': pd.Series(dtype='string'),
    'algo': pd.Series(dtype='string'),
    'length': pd.Series(dtype='int'),
    'f1': pd.Series(dtype='object'),
    'roc_auc': pd.Series(dtype='object'),
    # 'latencies_acc': pd.Series(dtype='object'),
    # 'latencies_rej': pd.Series(dtype='object'),
    'retrain': pd.Series(dtype='int'),
    'training_time_total': pd.Series(dtype='float'),
    'training_time_single': pd.Series(dtype='float'),
    'training_cpu_time': pd.Series(dtype='float'),
    'training_counter': pd.Series(dtype='int'),
    'inference_time_total': pd.Series(dtype='float'),
    'inference_time_single': pd.Series(dtype='float'),
    'inference_cpu_time': pd.Series(dtype='float'),
    'inference_counter': pd.Series(dtype='int'),
})

for i, row in stats_df.iterrows():
    print(i)
    temp_df = pd.read_csv(os.path.join(row['folder_path'], row['csv_path']), index_col=0)
    temp_df = temp_df.drop_duplicates(subset='minute', keep="last")
    retrain_list = temp_df['retrain'].tolist()
    retrain = [i for i in retrain_list if i==True]
    latencies_df = pd.read_csv(row['latencies_path'], header=None, names=['latency', 'reject'], dtype={'latency': int, 'reject': str})
    latencies_acc = latencies_df[latencies_df['reject'].isin(["0", "True"])]['latency'].tolist()
    latencies_acc_list[row['algo']] = latencies_acc
    latencies_rej = latencies_df[latencies_df['reject'].isin(["1", "False"])]['latency'].tolist()
    latencies_rej_list[row['algo']] = latencies_rej
    # if temp_df['minute'].max() == 480:
    # if i != 3:
    #     values_df_1.loc[len(values_df_1)] = [row['folder_path'], row['algo'], temp_df['minute'].max(), temp_df['f1_score'].tolist()[:-5], temp_df['roc_auc'].tolist()[:-5], row['num_retrain'], row['training_time_total'], row['training_time_single'], row['training_cpu_time'], row['training_counter'], row['inference_time_total'], row['inference_time_single'], row['inference_cpu_time'], row['inference_counter']]
    # else:
    values_df_1.loc[len(values_df_1)] = [row['folder_path'], row['algo'], temp_df['minute'].max(), temp_df['f1_score'].tolist(), temp_df['roc_auc'].tolist(), row['num_retrain'], row['training_time_total'], row['training_time_single'], row['training_cpu_time'], row['training_counter'], row['inference_time_total'], row['inference_time_single'], row['inference_cpu_time'], row['inference_counter']]

# print(values_df_1.iloc[13]['path'])
# values_df_1.drop([13], inplace=True)
values_df_1.reset_index(inplace=True, drop=True)
values_df_1.to_csv('values_all_minmax_latencies.csv')
values_df_1